In [1]:
# specify id
y_id = 0

In [2]:
# import module
import sys
sys.path.insert(0, '../')
from utils.training_utils import *
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import mean_absolute_error, make_scorer
from xgboost import XGBRegressor

In [3]:
# specify parameters
params = {
    'booster': ['gbtree', 'dart'],
    'n_estimators': [50, 100, 500],
    'max_depth': [3, 6],
    'subsample': [0.25, 0.5, 0.75],
    'learning_rate': [0.1, 0.05, 0.01], 
    'tree_method': ['hist', 'auto']
}
params_test = {
    'booster': ['dart'],
    'n_estimators': [50],
    'max_depth': [3],
    'subsample': [0.5],
    'learning_rate': [0.05]
}

In [4]:
# load datas
test_x, train_x, train_y = load_data(y_id)
print(test_x.shape, train_x.shape, train_y.shape)

(2500, 10000) (47500, 10000) (47500,)


In [5]:
# pick only important data
idx = []
with open('../29/adaboost' + str(y_id) + '_feature.csv', 'r') as f:
    i = 0
    for lines in f:
        importance = float(lines.replace('\n', '').split(',')[y_id])
        if(np.abs(importance) > 1e-9):
            idx.append(i)
        i += 1
train_x = train_x[:, idx]
test_x = test_x[:, idx]
print(train_x.shape)

(47500, 87)


In [6]:
# create GridSearchCV
model = GridSearchCV(estimator=XGBRegressor(verbosity=2, n_jobs=8), 
                     param_grid=params, 
                     scoring=make_scorer(mean_absolute_error),
                     cv=3,
                     verbose=20,
                     n_jobs=4,
                     return_train_score=True)

In [ ]:
# train
model.fit(train_x, train_y)

Fitting 3 folds for each of 216 candidates, totalling 648 fits


[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.


In [ ]:
# write files
write_prediction('train_y0_1.txt', 'w', model.predict(train_x).reshape((47500, 1)).astype('str'))
write_prediction('test_y0_1.txt', 'w', model.predict(test_x).reshape((2500, 1)).astype('str'))

In [ ]:
print(err1_calc(model.predict(train_x), train_y, y_id))